In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from collections import defaultdict
from datetime import datetime
import pymongo
import pytz

import pandas as pd
from matplotlib import pylab as plt
import seaborn as sns

from bot import config
from bot import database
import numpy as np

In [ ]:
db = database.Database()

In [ ]:
def change_tz_to_msk(d):
    return d.astimezone(pytz.timezone("Europe/Moscow"))


def user_to_str(user_dict):
    text = ""
    text += f"{user_dict['_id']}:\n"
    for k in ["username", "first_name", "last_name", "last_interaction", "first_seen", "current_chat_mode", "n_used_tokens", "token_balance"]:
        if k in user_dict:
            v = user_dict[k]
            if isinstance(v, datetime):
                v = change_tz_to_msk(v)
                v = v.strftime('%H:%M:%S %d-%m-%Y')
            text += f"  - {k}: {v}\n"

    return text


def dialog_to_str(dialog_dict):
    text = ""
    
    user_id = dialog_dict["user_id"]
    user_dict = db.user_collection.find_one({"_id": user_id})
    
    text += user_to_str(user_dict)
    text += "\n"
    
    text += f"Chat mode: {dialog_dict['chat_mode']}\n\n"
    text += "Messages:\n"
    for message in dialog_dict["messages"]:
        text += f"U: {message['user']}\n"
        text += f"B: {message['bot']}\n"
        text += "\n"
        
    return text


def payment_to_str(payment_dict):
    text = ""
    
    text += f"Status: {payment_dict['status']}\n"
    date = change_tz_to_msk(payment_dict["created_at"])
    text += f"Date: {date}\n"
    text += f"Amount: {payment_dict['amount']}\n\n"
    
    user_id = payment_dict["user_id"]
    user_dict = db.user_collection.find_one({"_id": user_id})
    
    text += user_to_str(user_dict)
    text += "\n"
    
    return text

### N total users

In [ ]:
n_users = db.user_collection.count_documents({})
print(f"N total users: {n_users}")

### Install graph

In [ ]:
query = db.user_collection.find({})

date_to_n_installs = defaultdict(int)
for user_dict in list(query):
    date_to_n_installs[user_dict["first_seen"].date()] += 1

date_to_n_installs_tuples = list(date_to_n_installs.items())
date_to_n_installs_tuples = sorted(date_to_n_installs_tuples, key=lambda x: x[0])

df = pd.DataFrame(data={
    "dates": [x[0] for x in date_to_n_installs_tuples],
    "installs": [x[1] for x in date_to_n_installs_tuples]
})

fig, ax = plt.subplots(1, 1, figsize=(15, 5))
sns.barplot(x='dates', y='installs', data=df, color="steelblue")

xticks = ax.get_xticks()
xticklabels = [x.get_text() for x in ax.get_xticklabels()]
_ = ax.set_xticks(xticks, xticklabels, rotation=90)

### Install graph (cumsum)

In [ ]:
query = db.user_collection.find({})

date_to_n_installs = defaultdict(int)
for user_dict in list(query):
    date_to_n_installs[user_dict["first_seen"].date()] += 1

date_to_n_installs_tuples = list(date_to_n_installs.items())
date_to_n_installs_tuples = sorted(date_to_n_installs_tuples, key=lambda x: x[0])

df = pd.DataFrame(data={
    "dates": [x[0] for x in date_to_n_installs_tuples],
    "installs": np.cumsum([x[1] for x in date_to_n_installs_tuples])
})

fig, ax = plt.subplots(1, 1, figsize=(15, 5))
sns.barplot(x='dates', y='installs', data=df, color="steelblue")

xticks = ax.get_xticks()
xticklabels = [x.get_text() for x in ax.get_xticklabels()]
_ = ax.set_xticks(xticks, xticklabels, rotation=90)

### N total messages from users

In [ ]:
query = db.dialog_collection.find({})

n_total_messages_from_users = 0
for dialog_dict in query:
    n_total_messages_from_users += len([1 for messages in dialog_dict["messages"]])
    
print(f"N total messages from users: {n_total_messages_from_users}")

### Message graph

In [ ]:
query = db.dialog_collection.find({})

date_to_n_messages = defaultdict(int)
for dialog_dict in query:
    for message in dialog_dict["messages"]:
        date_to_n_messages[message["date"].date()] += 1

In [ ]:
date_to_n_messages_tuples = list(date_to_n_messages.items())
date_to_n_messages_tuples = sorted(date_to_n_messages_tuples, key=lambda x: x[0])

df = pd.DataFrame(data={
    "dates": [x[0] for x in date_to_n_messages_tuples],
    "messages": [x[1] for x in date_to_n_messages_tuples],
})

fig, ax = plt.subplots(1, 1, figsize=(15, 5))
sns.barplot(x='dates', y='messages', data=df, color="steelblue")

xticks = ax.get_xticks()
xticklabels = [x.get_text() for x in ax.get_xticklabels()]
_ = ax.set_xticks(xticks, xticklabels, rotation=90)

### Money graph

In [ ]:
query = db.payment_collection.find({"status": "paid"})

date_to_money = defaultdict(int)
for payment_dict in list(query):
    amount = payment_dict["amount"]
    if payment_dict["currency"] == "RUB":
        amount /= 77
        
    date_to_money[payment_dict["created_at"].date()] += amount

date_to_money_tuples = list(date_to_money.items())
date_to_money_tuples = sorted(date_to_money_tuples, key=lambda x: x[0])

df = pd.DataFrame(data={
    "dates": [x[0] for x in date_to_money_tuples],
    "money": [x[1] for x in date_to_money_tuples]
})

fig, ax = plt.subplots(1, 1, figsize=(15, 5))
sns.barplot(x='dates', y='money', data=df, color="steelblue")

xticks = ax.get_xticks()
xticklabels = [x.get_text() for x in ax.get_xticklabels()]
_ = ax.set_xticks(xticks, xticklabels, rotation=90)

print(f"Total money: {sum(date_to_money.values())}")

### Last registered users

In [ ]:
print(f"5 last registered users:\n")

query = db.user_collection.find().sort('first_seen', -1).limit(5)
for user_dict in query:
    print(user_to_str(user_dict))
    print("_" * 50, "\n")

### Last successful payments

In [ ]:
print(f"Last successful payments:\n")

query = db.payment_collection.find({"status": "paid"}).sort("created_at", -1).limit(5)
for payment_dict in query:
    print(payment_to_str(payment_dict))
    print("_" * 50, "\n")

### Last created payments

In [ ]:
print(f"Last created payments:\n")

query = db.payment_collection.find({}).sort("created_at", -1).limit(5)
for payment_dict in query:
    print(payment_to_str(payment_dict))
    print("_" * 50, "\n")

### Last dialogs

In [ ]:
query = db.dialog_collection.find({"messages": {"$ne": []}}).sort('start_time', -1).limit(30)
for dialog_dict in query:
    print(dialog_to_str(dialog_dict))
    print("_" * 50, "\n")